In [3]:
!gdown --id 1w9K_XtKahNSITx-NE2K0BB_zlCMmfHke # 요약
!gdown --id 1jLZlWGvcGH9TCiKzQGHpsX1dXc6lkvLM # 번역

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1w9K_XtKahNSITx-NE2K0BB_zlCMmfHke
To: /content/summary.zip
100% 1.36G/1.36G [00:05<00:00, 244MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1jLZlWGvcGH9TCiKzQGHpsX1dXc6lkvLM
To: /content/translate.zip
100% 6.75G/6.75G [01:01<00:00, 109MB/s]


In [8]:
!pip install git+https://github.com/huggingface/datasets.git@master # huggingface 데이터 다운로드

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/datasets.git (to revision master) to /tmp/pip-req-build-3729vav2
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-3729vav2
     |████████████████████████████████| 212 kB 5.0 MB/s 
     |████████████████████████████████| 140 kB 41.9 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 37.8 MB/s 
     |████████████████████████████████| 127 kB 55.5 MB/s 
     |████████████████████████████████| 271 kB 23.6 MB/s 
     |████████████████████████████████| 144 kB 50.6 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Created wheel for datasets: filename=datasets-2.2.3.dev0-py3-none-any.whl size=354908 sha256=cad865d94b8feeb5251c551d681acf2ffabaffe359d41ed96741e04f581f8709
  Stored in dir

In [9]:
!pip install transformers # transformer 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 37.4 MB/s 


In [10]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MBartModel, MBartConfig ,PretrainedConfig
from transformers import (MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer)
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import BartForConditionalGeneration

In [12]:
# -*- coding: utf-8 -*-
"""파이프라인.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/117VZ3jRaFdpP4hn3mmYFdacAcK-96ZUN
"""

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MBartModel, MBartConfig ,PretrainedConfig
from transformers import (MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer)
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import BartForConditionalGeneration

class start_pipeline:
    def __init__(self,article, path_t, path_s):
        self.article = article,                 # 작성한 원문을 받아온다.
        self.path_t = '/content/translate.zip'       # pre-trained 된 MBart 모델 불러오는 경로
        self.path_s = '/content/summary.zip'       # pre-trained 된 KoBART 모델 불러오는 경로 

    # 원문(영어) > 번역문(한글)   
    def run_translate(self, article, path):
        model_translate = MBartForConditionalGeneration.from_pretrained(path_t)
        tokenizer_translate = tokenizer.from_pretrained(path_t)

        inputs = tokenizer_translate(article, return_tensors="pt")
        translated_tokens = model_translate.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"])
        translate = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

        return translate

#######################################################################################################################



    # 한글 >> 한글(요약)
    def run_summarization(self, translate, model, tokenizer):
        tokenizer_summary = PreTrainedTokenizerFast.from_pretrained(path_s)
        model_summary = BartForConditionalGeneration.from_pretrained(path_s)
        article = ''.join(str(e) for e in translate[0][1:-2])
        article = article.replace('\n', ' ') # 엔터를 공백으로
        
        raw_input_ids = tokenizer_summary.encode(article)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

        summary_ids = model_summary.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
        result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

        return result

